In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import re

In [2]:
class RiaParser(object):
    def __init__(self, date=None, dateend=None):
        """
        На вход ожидается строка вида '2018-10-15'
        Или кортеж вида ('2018-10-14', '2018-10-15')
        """
        if date is None:
            self.date = self(datetime.now().date())
        else:
            if dateend is None:
                self.date = date
            else:
                self.date = (date, dateend)
        self.link = "https://ria.ru/archive/"
        self.links = []

    def _get_day_links(self, day):

        # Я увидела, что с 1 раза не всегда открывается, поэтому решила дать ему 3 попытки
        while True:
            try:
                # выкачиваем ссылки на новейшие статьи переданный день
                resp = requests.get(self.link + day + '/')
                # print('\n', resp.status_code)
                bss = BeautifulSoup(resp.text, "html5lib")
                mega_link = bss.find("a", class_="b-pager__button m-button-more")[
                    "data-ajax"]  # мегассыль на "загрузить еще"
                date = re.search(r'\d{8}', mega_link)[0]
                break
            except:
                time.sleep(0.3) 

        # time = re.findall(r'\d{6}', mega_link)[1]
        # выкачивание ссылок на статьи
        for element in bss.find_all("div", class_="b-list__item"):
            link = element.find("a")["href"]
            date = re.search(r'\d{8}', link)[0]  # проверка нужной даты в ссылке
            if date == day:
                self.links.append(link)
                print('Внимание! Обнаружено ', len(self.links), ' статей!\r', end='')
        # идем дальше в архив по нужному дню по ссылке, которую мы добыли
        while date == day:
            
            # Я увидела, что с 1 раза не всегда открывается, поэтому решила дать ему 3 попытки
            for i in range(3):
                try:
                    resp = requests.get('https://ria.ru/' + mega_link)
                    # print('\n', resp.status_code)
                    bss = BeautifulSoup(resp.text, "html5lib")
                    mega_link = bss.find("a", class_="b-pager__button m-button-more")["data-ajax"]
                    break
                except:
                    mega_link = None
                    time.sleep(0.3)
            # Если мы в конце концов ничего не скачали, то это означает, что ссылок больше нет
            if mega_link is None:
                break
                
            date = re.search(r'\d{8}', mega_link)[0]
            # выкачивание ссылок на статьи
            for element in bss.find_all("div", class_="b-list__item"):
                link = element.find("a")["href"]
                date = re.search(r'\d{8}', link)[0]  # проверка нужной даты в ссылке
                if date == day:
                    self.links.append(link)
                    print('Внимание! Обнаружено ', len(self.links), ' статей!\r', end='')
        print()

    def get_links(self):
        if isinstance(self.date, str):
            print('Обрабатываю ', self.date)
            self._get_day_links(re.sub('-', '', self.date))
        if isinstance(self.date, tuple):
            day = datetime.strptime(self.date[0], '%Y-%m-%d').date()
            delta = timedelta(days=1)
            end = datetime.strptime(self.date[1], '%Y-%m-%d').date()
            while day <= end:
                print('Обрабатываю ', str(day))
                self._get_day_links(re.sub('-', '', str(day)))
                day += delta

    def _get_article(self, link):
        """
        Парсим название и тело статьи
        """
        resp = requests.get(link)
        bs = BeautifulSoup(resp.text, "html5lib")
        try:
            title = bs.h1.text.replace("\xa0", " ")
        except:
            title = "Без названия"
        class_name = "b-article__body js-mediator-article mia-analytics"
        article = ""
        article = " ".join([para.text.replace("\xa0", " ") for para in bs.find("div", class_=class_name).find_all("p")])
        return title + "\n" + article + "\n"

    def get_all_articles(self):
        """
        Проходим по всем статьям и парсим их
        """
        links = []  # список статей
        self.allarticles = ""
        count = 0  # это показатель, сколько статей обработано (т.к. все достаточно медленно)
        success = 0
        for link in self.links:
            lk = "https://ria.ru" + link
            # Не все статьи имеют такой вид, поэтому я тоже написала try, чтобы откинуть неправильные статьи
            try:
                self.allarticles += self._get_article(lk)
                success += 1
            except:
                pass
            count += 1
            print('Обработано статей:', count, ' Успешно:', success, '\r', end='')

    
    def to_txt(self, filename):
        """
        Записываем в файл
        """
        with open(filename, 'w', encoding='utf8') as f:
            f.write(self.allarticles)

In [3]:
my = RiaParser('2018-08-31', '2018-09-01')

In [4]:
%%time
my.get_links()

Обрабатываю  2018-08-31
Внимание! Обнаружено  380  статей!
Обрабатываю  2018-09-01
Внимание! Обнаружено  660  статей!
Wall time: 19.6 s


In [5]:
%%time
my.get_all_articles()

Wall time: 5min 5s 660  Успешно: 642 


In [6]:
%%time
my.to_txt('data.txt')

Wall time: 8.01 ms
